# Python Intro - Final Exercise


In [67]:
import pyodbc
import pandas as pd
import numpy as np

In [68]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=collage;Trusted_Connection=yes;')


In [69]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
departments = pd.read_sql("SELECT * FROM Departments;",conn)
teachers = pd.read_sql("SELECT * FROM Teachers;",conn)
courses = pd.read_sql("SELECT * FROM Courses;",conn)
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn)

# Questions
 
 
## Q1. Count the number of students on each department

In [82]:
Q1 = classrooms.merge(courses, on = 'CourseId').merge(departments, on = 'DepartmentId')
Q1.groupby('DepartmentName').agg({'StudentID': pd.Series.nunique})


,StudentID
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [192]:
Q2 = classrooms.merge(courses[courses.DepartmentId == 1], on = 'CourseId')
Q2a = Q2.groupby('CourseName').agg({'StudentID':pd.Series.nunique})
Q2a.reset_index(inplace=True)
tt = sum(Q2a.StudentID)
Total = pd.DataFrame([('Total',tt)], columns=['CourseName','StudentID'])
Q2a.append(Total)

,CourseName,StudentID
0,Advanced English,24
1,English Begginers,32
2,Proffesional English,38
0,Total,94


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [282]:
Q3 = pd.DataFrame(classrooms.merge(courses[courses.DepartmentId == 2], on = 'CourseId'))
Q3a= Q3.groupby('CourseName').agg({'StudentID' :'count'})
Q3a.columns=['Total']
Q3b = Q3a.copy()
Q3b['Size'] = ''
Q3b.loc[(Q3b['Total']<= 22),'Size']='Small'
Q3b.loc[(Q3b['Total']> 22),'Size']='Big'
Q3b.groupby('Size').agg({'Size':['count']})

,Size
,count
Size,
Big,10
Small,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [220]:
Q5=students.groupby('Gender').agg({'StudentID':pd.Series.nunique})
Q5

,StudentID
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [267]:
Q5 = pd.merge(students, classrooms ,on = 'StudentID', how = 'left').merge(courses, on = 'CourseId', how = 'left')
Q5a = Q5.groupby(['CourseName','Gender'] ).agg({'Gender' :'count'})
Q5b = Q5a.copy()
Q5b.colums = ['Totalgender']
Q5b['Totalclass'] = Q5b.groupby('CourseName').transform('sum')
Q5b['Precent'] = Q5b.Gender/Q5b.Totalclass
Q5b = Q5b[Q5b.Precent >0.7]
Q5b

C:\Users\rafic\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


,,Gender,Totalclass,Precent
CourseName,Gender,,,
Sculpture,F,17,24,0.708333
Tenis,F,13,18,0.722222


## Q6. For each department, how many students passed with a grades over 80?

In [340]:
Q6 = pd.merge(classrooms, courses, on = 'CourseId').merge(departments, on ='DepartmentId')
Q6a = Q6.copy()
Q6a = Q6a[Q6a.degree>80]
Q6a.groupby(['DepartmentName']).agg({'StudentID' :'count'})

,StudentID
DepartmentName,
Arts,68
English,43
Science,171
Sport,41


## Q7. For each department, how many students passed with a grades under 60?

In [280]:
Q7 = pd.merge(classrooms, courses, on = 'CourseId').merge(departments, on ='DepartmentId')
Q7a = Q7.copy()
Q7a = Q7a[Q7a.degree<60]
Q7a.groupby(['DepartmentName']).agg({'StudentID' :pd.Series.nunique})

,StudentID
DepartmentName,
Arts,3
English,2
Science,7
Sport,1


## Q8. Rate the teachers by their average student's grades (in descending order).

In [338]:
Q8 = pd.merge(courses, classrooms ,on = 'CourseId', how = 'left').merge(teachers, on = 'TeacherId', how = 'left')
Q8a = Q8.groupby(['TeacherId', 'FirstName', 'LastName']).agg({'degree' :'mean'})
Q8b = Q8a.sort_values('degree', ascending=False)
Q8b

,,,degree
TeacherId,FirstName,LastName,
22,Darick,Hess,81.432703
3,Jordyn,Baron,80.729494
5,Heather,Metcalf,80.440242
20,Zachary,Healy,79.365867
1,Kaylea,Monzingo,79.365537
18,Daniel,Bland,78.908281
24,Keegan,Price,78.814631
15,Conor,Mckittrick,78.699492
23,Maxwell,Romkee,78.673509


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [320]:
Q9 = pd.merge(courses, classrooms ,on = 'CourseId', how = 'left').\
merge(teachers, on = 'TeacherId', how = 'left').\
merge(departments, on = 'DepartmentId', how = 'left')
Q9a = Q9[['DepartmentName', 'CourseName', 'FirstName', 'StudentID']]
Q9b = Q9a.groupby(['DepartmentName', 'CourseName', 'FirstName']).agg({'StudentID' :'count'})
Q9b.rename(columns={'StudentID':'NumOfStudents'}, inplace=True)
Q9b

NumOfStudents
DepartmentName CourseName           FirstName                
Arts           History of Arts       Caroline              32
               Literature            Daniel                40
               Musicology            Madison               30
               Painting              Katerina              34
               Sculpture             Robert                24
               Theater               Ernest                26
English        Advanced English      Conor                 24
               English Begginers     Conor                 32
               Proffesional English  Conor                 38
Science        Algebra               Alexander             12
               Biology               Jordyn                36
               Chemistry             Jeffrey               24
               Computer Science      Kaylea                38
               Data Science          Mackenzie             30
               Genetics              Terran                38
               Geometry              Alexander             14
               Java                  Darick                16
               Matematics Advanced   Alexander             10
               Mathematics 101       Amanda                18
               Neurobiology          Terran                24
               Physics               Alexander             12
               Python                Terran                24
               R                     Tanner                34
               SQL                   Zachary               24
               Statistics            Heather               24
Sport          Basketball            Katherine              0
               Football              Cortnee                0
               Gymnastics            Maxwell               24
               Swimming              Mikayla               18
               Tenis                 Keegan                18
               Volleyball            Kaitlin               36

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [467]:
Q10 = pd.merge(students, classrooms ,on = 'StudentID', how = 'left')
Q10a = Q10[['StudentID', 'FirstName', 'LastName', 'CourseId','degree']]
Q10b = Q10a.groupby(['StudentID' , 'FirstName', 'LastName']).agg({'degree' :'mean','CourseId':'count'})
Q10b.rename(columns={'CourseId':'NumOfCourses', 'degree': 'TotalAVG'}, inplace=True)

avg1= pd.merge(students, classrooms ,on = 'StudentID', how = 'left').\
merge(courses ,on = 'CourseId', how = 'left')

avg2 = avg1[['StudentID', 'CourseId','DepartmentId','degree']]
avg3 = avg2.groupby([ 'StudentID', ]).agg({'degree' :'mean','DepartmentId':'sum' })

avgByDep1 = avg3[avg3['DepartmentId']== 1]
avgByDep2 = avg3[avg3['DepartmentId']== 2]
avgByDep3 = avg3[avg3['DepartmentId']== 3]
avgByDep4 = avg3[avg3['DepartmentId']== 4]

avgByDep1.rename(columns={'degree': 'AVGbyDep1'}, inplace=True)
avgByDep2.rename(columns={'degree': 'AVGbyDep2'}, inplace=True)
avgByDep3.rename(columns={'degree': 'AVGbyDep3'}, inplace=True)
avgByDep3.rename(columns={'degree': 'AVGbyDep3'}, inplace=True)

res = pd.merge(Q10b, avgByDep1, on = 'StudentID', how = 'left').\
merge(avgByDep2,on = 'StudentID', how = 'left') 

res = res[['StudentID', 'FirstName','LastName','degree', 'avgByDep1', 'avgByDep2' ]]

#Q10b.head(10)

KeyError: "None of [Index(['StudentID', 'FirstName', 'LastName', 'degree', 'avgByDep1',\n       'avgByDep2'],\n      dtype='object')] are in the [columns]"